# Clean up output file
* Invalid emails: 
    * From element
    * No Sent time
* Remove items from CC: field
* Remove items from To: field (if there is no email and no name)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import pandas as pd
import numpy as np
import regex as rex
import pandas as pd
from datetime import datetime
from dateutil import parser
from dateutil.tz import gettz
from unidecode import unidecode
import process_to_library as tolib
import standardize_library as standlib

In [76]:
df = pd.read_csv("output_attr_stats.csv.gz")

In [85]:
df.shape

(35942, 28)

In [86]:
# Remove all rows with negative timestamps
df1 = df[df['timestamp'] > 0 ]
df1.shape

(35357, 28)

In [88]:
df2 = df1[df1.From != 'invalid']
df2.shape

(35115, 28)

In [122]:
df2 = df2[pd.isnull(df4.From) == False]
df2.shape

(35106, 28)

CC elements to remove: 
* ['850)545-2095edward.kring@talgov.comsentfrommyiphonethanks'
* "bryant ==> bryan  # remove quote)
* "mike(firedept" ==> mike
*  bellamy  # (what is this?)
* cherie(planning => cherie   # why wasn't parenthesis removed?
    * 51733_fn_34-5-ScottMaddox5_ln_54108.txt," Thomas, Debra (Planning) <Debra.Thomas@talgov.com"," Thursday, November 7, 2013 10:27 AM", Mayo      r & City Commissioners," Bryant, Cherie (Planning); McDonald,Earnest; Faris, Alison",, Palmer Avenue Block Party Ceremony Time, image001.      jpg,,False,False,True,False,False,"

In [196]:
def remove_invalid_from_list(my_list):
    # Construct new To: list with invalid cc: elements removed. Do not count them. 
    # Remove NaN as well.
    new_my_list = []
    for lst in my_list:
        split = lst.split(";")
        new_split = []
        for el in split:
            if not rex.match('.*invalid', el, flags=rex.I):
                new_split.append(el)
            # I do not expect spaces
            if rex.match('.*\s', el):
                print("space match: ", el)
        new_my_list.append(";".join(new_split))
    return new_my_list

In [197]:
to_list = remove_invalid_from_list(df2.To.values)
cc_list = remove_invalid_from_list(df2.CC.values)
len(to_list), len(cc_list)

(35106, 35106)

In [192]:
# Check that there are no "invalid"s left
for el in cc_list:
    if rex.match('.*invalid', el):
        print(el)

In [172]:
df3 = df2.copy()

In [193]:
for el in df3.values:
    try:
        if rex.match(r'.*INVALI', el):
            print(el)
    except:
        cols = df3.columns
        for c,e in zip(cols, el):
            print(f"{c}: {e}")
        break

filenm: 29142_fn_10-4-Cascade-2015-1-0_ln_42056.txt
From: shaneamoniz
Sent: Friday, May 08, 2015 11:38 AM
To: brooks@culpeppercc.com
CC: rgrindler@101tally.com;davidrosenfeld
Bcc: nan
Subject: Edison RFI's
Attachments: RFI 008 Response.pdf; RFI 042 Response.pdf; RFI 043 Response.pdf; RFI 044
Importance: nan
isThread: False
isAutoMessage: False
isDisplacement: False
hasAllCapLine: False
hasBadDate: True
Body: ['Response.pdf', 'Brooks, attached is our response to RFIs 008, 042, 043, and 044.', 'Shane A. Moniz, AIA', 'Architect', 'Conn & Associates, Inc., Architects', '1960-C Buford Boulevard', 'Tallahassee, Florida 32308', 'p 850-878-8784', 'm 850-445-4056', 'smoniz@connandassociates.com', 'NUMBER:', 'REQUEST FOR INFORMATION 003', 'Date: March 2, 2015 PROJECT The Edison Restaurant Cascades Park', 'REFERENCE Project 0500', 'To: Shane Moniz']
timestamp: 1431081480.0
date_sent: 2015-05-08
time_sent: 10:38
cc: Ryan Grindler  rgrindler@101tally.com ; Rosenfeld, David
nb_words: 55
nb_chars: 33

In [190]:
df3.loc[:,'To'] = to_list
df3.loc[:,'CC'] = cc_list

In [189]:
# From: field should only contain \w characters. Identify those that don't
count=0
for el in df3.CC.values:
    if  not rex.match(r'\A[A-Za-z0-9;]*\Z', el) and not rex.match(r'.*@',  el):
        # print(el)
        count += 1
    if rex.match(r'.*invalid\b',  el):
        print(el)
print("nb invalid_nan: ", count)

nb invalid_nan:  0


In [195]:
df3.to_csv("output_without_invalid_mails.csv", index=0)